In [62]:
import re
import numpy as np
import pandas as pd
from typing import Dict, Tuple

def parse_metrics_table(text: str) -> Dict[str, Tuple[float, float, float]]:
    """
    
    format：
    ClassID      Recall    Precision    AP@0.5
    ---------  --------  -----------  --------
    MTL            0.37         0.40      0.22
    ...
    """
    metrics = {}
    
    # 按行分割
    lines = text.strip().split('\n')
    
    # 
    for line in lines:
        line = line.strip()
        if not line or '---' in line or 'ClassID' in line:
            continue
        
        # 
        parts = re.split(r'\s+', line)
        if len(parts) >= 4:
            class_id = parts[0]
            try:
                recall = float(parts[1])
                precision = float(parts[2])
                ap50 = float(parts[3])
                metrics[class_id] = (recall, precision, ap50)
            except ValueError:
                print(f"warning: {line}")
    
    return metrics

def calculate_macro_micro_metrics(metrics: Dict[str, Tuple[float, float, float]], 
                                  gt_counts: Dict[str, int]) -> Dict[str, float]:
    """
    
    """
    results = {}
    
    # 
    all_classes = list(gt_counts.keys())
    total_gt = sum(gt_counts.values())
    
    # 
    recalls = []
    precisions = []
    f1_scores = []
    
    total_tp = 0
    total_pred = 0
    
    for class_id in all_classes:
        if class_id in metrics:
            recall, precision, _ = metrics[class_id]
        else:
            recall, precision = 0.0, 0.0  #         
        recalls.append(recall)
        precisions.append(precision)
        
        #         if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0.0
        f1_scores.append(f1)
        
        #         tp = recall * gt_counts[class_id]
        total_tp += tp
        
        if precision > 0:
            pred = tp / precision
        else:
            pred = 0
        total_pred += pred
    
    #    results['Macro-P'] = np.mean(precisions) if precisions else 0
    results['Macro-R'] = np.mean(recalls) if recalls else 0
    results['Macro-F1'] = np.mean(f1_scores) if f1_scores else 0
    
    #    if total_pred > 0:
        results['Micro-P'] = total_tp / total_pred
    else:
        results['Micro-P'] = 0
    
    if total_gt > 0:
        results['Micro-R'] = total_tp / total_gt
    else:
        results['Micro-R'] = 0
    
    if results['Micro-P'] + results['Micro-R'] > 0:
        results['Micro-F1'] = 2 * results['Micro-P'] * results['Micro-R'] / (results['Micro-P'] + results['Micro-R'])
    else:
        results['Micro-F1'] = 0
    
    return results

def main():
    #     input_text = """

Class      Recall    Precision    AP@0.5
-------  --------  -----------  --------
MTL        0.5027       0.1479    0.1914
TEE        0.7368       0.5185    0.6546
BND        0.5732       0.1957    0.2286
CRC        0.4181       0.0628    0.1344
BRN        0.5556       0.2941    0.4434
GWA        0.1734       0.1398    0.0502
SWA        0.4078       0.0842    0.0895
ESP        0.7778       0.3043    0.4765
VAL        0.8696       0.6667    0.7032
FLA        0.5000       0.0541    0.2794
CAS        0.3200       0.4444    0.2143
SLE        0.7931       0.2754    0.6854"""
    
    # 
    gt_counts = {
        'MTL': 29720,
        'CRC': 573,
        'GWA': 4135,
        'SWA': 3423,
        'BND': 708,
        'SLE': 58,
        'BRN': 47,
        'TEE': 36,
        'CAS': 25,
        'VAL': 23,
        'ESP': 9,
        'FLA': 4
    }
    
    # 
    metrics = parse_metrics_table(input_text)
    
    print("")
    print(f"{'category':<5} {'Recall':<8} {'Precision':<10} {'AP@0.5':<8}")
    print("-" * 35)
    for class_id, (recall, precision, ap50) in metrics.items():
        print(f"{class_id:<5} {recall:<8.4f} {precision:<10.4f} {ap50:<8.4f}")
    print()
    
    #n    results = calculate_macro_micro_metrics(metrics, gt_counts)
    
    #     print(":")
    print("=" * 40)
    print(f"{'':<15} {'':<10}")
    print("-" * 40)
    for key, value in results.items():
        print(f"{key:<15} {value:.4f}")
    
    # mAP（）
    ap50_values = [ap50 for _, _, ap50 in metrics.values()]
    mAP = np.mean(ap50_values) if ap50_values else 0
    print(f"{'mAP@0.5':<15} {mAP:.4f}")
    
    # 
    print("\n:")
    print(f": {sum(gt_counts.values())}")
    print(f": {len(gt_counts)}")
    
    # 
    data = []
    for class_id in gt_counts.keys():
        if class_id in metrics:
            recall, precision, ap50 = metrics[class_id]
        else:
            recall, precision, ap50 = 0, 0, 0
        
        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0
        
        data.append({
            'Class': class_id,
            'GT': gt_counts[class_id],
            'Recall': recall,
            'Precision': precision,
            'AP@0.5': ap50,
            'F1': f1
        })
    
    df = pd.DataFrame(data)
    print("\n:")
    print(df.to_string(index=False))

if __name__ == "__main__":
    main()

 Class      Recall    Precision    AP@0.5

类别    Recall   Precision  AP@0.5  
-----------------------------------
MTL   0.5027   0.1479     0.1914  
TEE   0.7368   0.5185     0.6546  
BND   0.5732   0.1957     0.2286  
CRC   0.4181   0.0628     0.1344  
BRN   0.5556   0.2941     0.4434  
GWA   0.1734   0.1398     0.0502  
SWA   0.4078   0.0842     0.0895  
ESP   0.7778   0.3043     0.4765  
VAL   0.8696   0.6667     0.7032  
FLA   0.5000   0.0541     0.2794  
CAS   0.3200   0.4444     0.2143  
SLE   0.7931   0.2754     0.6854  

result:
                       
----------------------------------------
Macro-P         0.2657
Macro-R         0.5523
Macro-F1        0.3323
Micro-P         0.1382
Micro-R         0.4601
Micro-F1        0.2126
mAP@0.5         0.3459

:
GT: 38761
Categories: 12

per category:
Class    GT  Recall  Precision  AP@0.5       F1
  MTL 29720  0.5027     0.1479  0.1914 0.228556
  CRC   573  0.4181     0.0628  0.1344 0.109198
  GWA  4135  0.1734     0.1398  0.0502 0.154